## librerias

In [1]:
import urllib.request
import requests
import sys                                                  # built in
from bs4 import BeautifulSoup
import pandas as pd
from pandas.io.json import json_normalize
import json                                                 # built in
import ast                                                  # built in
import numpy as np

import time

import pandas as pd

from multiprocessing import Pool

import os

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import ActionChains

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By


# set options to be headless, ..

options = Options()
options.add_argument('--blink-settings=imagesEnabled=false')

#options.add_argument('--headless')
#options.add_argument('--no-sandbox')
#options.add_argument('--disable-dev-shm-usage')


## codigo driver get url


In [3]:
# pagina de busqueda

url = 'https://play.google.com/store/apps/details?id=us.zoom.videomeetings&hl=es_PE&gl=US&showAllReviews=true'


In [4]:
driver = webdriver.Chrome(executable_path=r"C:\chromedriver_win32\chromedriver.exe", options=options)
driver.get(url)

In [5]:
# get html
page_detailed = driver.page_source

# close the driver
driver.quit()

# BS
detail_soup = BeautifulSoup(page_detailed)

In [6]:
# listings = detail_soup.find_all('div', 'card  edit_hover_class')  

listings = detail_soup.find_all('div', 'd15Mdf bAhLNe')    # UD7Dzf


In [7]:
len(listings)

520

## all features

In [8]:
RULES_SEARCH_PAGE = {
    'comentario': {'tag': 'div', 'class': 'UD7Dzf'},  
    'fecha': {'tag': 'span', 'class': 'p2TkOb'}, 
   'likes': {'tag': 'div', 'class': 'jUL89d y92BAb'}
}


# Third Generation :)
def extract_element(listing_html, params):
    # 1. Find the right tag
    if 'class' in params:
        elements_found = listing_html.find_all(params['tag'], params['class'])
    else:
        elements_found = listing_html.find_all(params['tag'])

    # 2. Extract the right element
    tag_order = params.get('order', 0)
    element = elements_found[tag_order]
        
    # 3. Get text
    if 'get' in params:
        output = element.get(params['get'])
    else:
        output = element.get_text()

    return output

In [9]:
for feature in RULES_SEARCH_PAGE:
    try:
        print(f"{feature}: {extract_element(listings[0], RULES_SEARCH_PAGE[feature])}")
    except:
        print(f"{feature}: empty")

comentario:  Es una aplicación muy buena en estos tiempos difíciles, para toda buena obra,gracias, no para dañar,me gusta mucho. De mucha ayuda gracias.
fecha: December 28, 2021
likes: 43


In [10]:
def extract_page_features(soup, rules):
    features_dict = {}
    for feature in rules:
        try:
            features_dict[feature] = extract_element(soup, rules[feature])
        except:
            features_dict[feature] = 'empty'
    
    return features_dict

In [11]:
features_list = []

for comment in listings:
            features = extract_page_features(comment, RULES_SEARCH_PAGE)
            features_list.append(features)

In [12]:
features_list[:3]

[{'comentario': ' Es una aplicación muy buena en estos tiempos difíciles, para toda buena obra,gracias, no para dañar,me gusta mucho. De mucha ayuda gracias.',
  'fecha': 'December 28, 2021',
  'likes': '43'},
 {'comentario': ' Deberían de optimizar más la interfaz para móviles ya que la de tablets esta mas cuidada que la de móviles partiendo de los diseño y vista de galería lo contrario que ocurre con meet',
  'fecha': 'January 4, 2022',
  'likes': '14'},
 {'comentario': ' Muy bueno pero los anuncios interfieren muchísimo en la calidad de este servicio tan bueno',
  'fecha': 'December 29, 2021',
  'likes': '38'}]

In [13]:
df = pd.DataFrame.from_dict(features_list)

df.shape

(520, 3)

In [14]:
df.head()

,comentario,fecha,likes
0,Es una aplicación muy buena en estos tiempos ...,"December 28, 2021",43
1,Deberían de optimizar más la interfaz para mó...,"January 4, 2022",14
2,Muy bueno pero los anuncios interfieren muchí...,"December 29, 2021",38
3,Ha sido de mucha ayuda. Pienso que en muchos ...,"December 4, 2021",128
4,Me gusta la calidad de fotos. Pero hay una di...,"November 23, 2021",91


In [15]:
import openpyxl

# insertar ruta de la carpeta donde se exportará el dataframe
df.to_excel(r"D:\PUCP\casos_analytics\zoom_reviews.xlsx", index = False)